In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
import os

In [4]:
import amstools

In [5]:
from amstools.lammps import write_in_lammps

In [6]:
from amstools.qatoms import QAtoms

In [61]:
structure = QAtoms.bulk("Al",cubic=True)*(5,5,5)
structure=structure.sample(frac=0.05, random_state=42).set(element='Li')

In [62]:
structure

QAtoms(symbols='Al475Li25', pbc=True, cell=[20.25, 20.25, 20.25], _selected_mask=..., 25 selected atoms)

In [63]:
T=500
steps=[
    
    {
        "type": "eq",
        "eq_type": "npt",             # 'npt', 'nvt', or 'nve'
        "thermo_freq": 250,
        "steps": 1000,
        "T_init": 2*T,                # Initial temperature for velocity creation
        "T1": T, "T2": T,          # Target temperature range
        "P1": 0, "P2": 0,              # Target pressure range (for NPT)
        "aniso_style": "iso",         # Pressure coupling style (for NPT)
        "seed": 12345,                # Random seed for velocity creation
        "dump": True,                 # Whether to dump trajectory
    },
    
    {
        "type": "mdmc",
        # "name": "my_mdmc_swap",
        "thermo_freq": 500,
        "steps": 50000,
        "T_MDMC": T,               # Temperature for Monte Carlo swap attempts
        "T1": T, "T2": T,        # Temperature for MD (NPT)
        "P1": 0, "P2": 0,              # Pressure for MD (NPT)
        "aniso_style": "aniso",
        # "elements": ["Ni", "Co", "Cr"], # Elements to be swapped, by default - all possible pairs
        "N": 100,                     # Perform M swap attempts every N steps
        "M": 100,                     # Number of swap attempts to perform
    },
                    
]

# Setup LAMMPS with FS-distilled model

In [65]:
cwd=os.getcwd()

In [67]:
potname=f"{cwd}/../3b-distillation/seed/1/saved_model.yaml"
potname=os.path.abspath(potname)
potname

'/home/users/lysogy36/PycharmProjects/GITHUB/grace-tutorial-fresh/3-foundation-models/3b-distillation/seed/1/saved_model.yaml'

In [68]:
write_in_lammps(structure=structure, potname=potname, 
                working_dir="lammps-grace-fs-dist",
                pairstyle='grace/fs',
                with_extrapolation_grade=True,
                steps=steps
                
               )
            

# Setup LAMMPS with reference finetuned GRACE-2L

In [75]:
grace_potname=f"{cwd}/../3a-finetuning/seed/1/final_model/"
grace_potname=os.path.abspath(grace_potname)
grace_potname

'/home/users/lysogy36/PycharmProjects/GITHUB/grace-tutorial-fresh/3-foundation-models/3a-finetuning/seed/1/final_model'

In [76]:
write_in_lammps(structure=structure, potname=grace_potname, 
                working_dir="lammps-grace-2L",
                pairstyle='grace',
                steps=steps
               )
            